# Config

In [1]:
# This script retrieves the list of organizations from the Open Data API of Barcelona.
# It uses the requests library to make API calls and pandas for data manipulation.
import requests
import os
import pandas as pd
from shapely import wkt

import numpy as np
import geopandas as gpd
from config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, CKAN_API_KEY, RAW_DATA_DIR
from io import StringIO

https://portaldades.ajuntament.barcelona.cat/ca/microdades/90dc3d6e-1c9a-4136-aed8-74bebb43e052

https://portaldades.ajuntament.barcelona.cat/ca/microdades/eea11ee9-cef5-41e8-841f-df2bbc809041

https://portaldades.ajuntament.barcelona.cat/ca/microdades/a45e1f19-5137-45bd-8979-645906fde55b

Revisar:
https://portaldades.ajuntament.barcelona.cat/ca/microdades/d8e40c96-9f1f-4fd3-86da-2baa1599616d

In [ ]:
# Base API setPoblació de Barcelona agregada per edat en grups de cinc anys segons el registre del Padró Municipal d'Habitants a data 1 de gener de cada any
tings
API_URL = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/microdata/export?"
headers = {'X-IBM-Client-Id': CKAN_API_KEY}

# Define datasets (unique keys)
DATASETS = {
    'Vehicles per type': {
        'id': '90dc3d6e-1c9a-4136-aed8-74bebb43e052',
        'resource_id': '344e1547-8e13-4452-8fa8-f362601a9d52',
        'year': 2024
    },
    'Population per continent 2025': {
        'id': 'eea11ee9-cef5-41e8-841f-df2bbc809041',
        'resource_id': 'ab539ff9-7133-47ba-9114-51a6aa855d7d',
        'year': 2025
    },
    'Population per age group 2025': {
        'id': 'a45e1f19-5137-45bd-8979-645906fde55b',
        'resource_id': 'c5af1fec-95bc-4f25-8adc-c0313cfe0144',
        'year': 2025
    },
    'Rent per household': {
        'id': 'd8e40c96-9f1f-4fd3-86da-2baa1599616d',
        'resource_id': 'fd12fd1f-5fe6-4642-9ace-34503c2a9dd5',
        'year': 2023
    },
}

# List of JSON URLs
json_urls = [
    "https://opendata-ajuntament.barcelona.cat/resources/bcn/EstadisticaPadro/pad/2025/2025_pad_mdbas_nacionalitat-continent_sexe.json",
    "https://opendata-ajuntament.barcelona.cat/resources/bcn/EstadisticaPadro/pad/2025/2025_pad_mdbas_edat-q.json",
    "https://opendata-ajuntament.barcelona.cat/resources/bcn/EstadisticaUnitatsAdministratives/BarcelonaCiutat_SeccionsCensals.json"
]

# Load data

## Read CSV using the API

In [ ]:

dfs = {}

for name, info in DATASETS.items():
    print(f"Fetching {name}...")
    params = {
        "id": info["id"],
        "idResource": info["resource_id"],
        "language": "ca",
        "fileformat": "CSV"
    }

    r = requests.get(API_URL, headers=headers, params=params)
    if r.status_code == 200:
        text = r.content.decode('utf-8')
        df = pd.read_csv(StringIO(text))
        dfs[name] = df
        print(f"  → Retrieved {len(df)} rows")
        df.to_csv(name.lower().replace(' ', '_') + ".csv", index=False)
    else:
        print(f"  ⚠️ Error {r.status_code}:")

print("\nExample preview:")
for k, v in dfs.items():
    print(f"\n{k}\n", v.head())

Fetching Vehicles per type...
  → Retrieved 6403 rows
Fetching Population per continent 2025...
  ⚠️ Error 500:
Fetching Population per age group 2025...
  ⚠️ Error 500:
Fetching Rent per household...
  ⚠️ Error 500:

Example preview:

Vehicles per type
     Any Codi_Districte Nom_Districte Codi_Barri Nom_Barri Seccio_censal  \
0  2024             01  Ciutat Vella         01  el Raval           001   
1  2024             01  Ciutat Vella         01  el Raval           001   
2  2024             01  Ciutat Vella         01  el Raval           001   
3  2024             01  Ciutat Vella         01  el Raval           001   
4  2024             01  Ciutat Vella         01  el Raval           001   

  Tipologia_parc  Nombre_vehicles  
0       Turismes              349  
1          Motos              146  
2    Ciclomotors              245  
3     Furgonetes               37  
4        Camions               47  


## Read JSON

In [4]:
def load_json_dataset(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch {url}: {response.status_code}")
    
    data = response.json()  # list of dictionaries
    df = pd.DataFrame(data)
    
    # Convert numeric columns if possible
    for col in ["Valor", "SEXE", "NACIONALITAT_CONTINENT", "EDAT_Q"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

# Load all JSONs
for url in json_urls:
    name = url.split("/")[-1].replace(".json", "")
    dfs[name] = load_json_dataset(url)
    print(f"{name} loaded with {len(dfs[name])} rows")

# Example preview
print(dfs["2025_pad_mdbas_nacionalitat-continent_sexe"].head())
print(dfs["2025_pad_mdbas_edat-q"].head())

2025_pad_mdbas_nacionalitat-continent_sexe loaded with 9275 rows
2025_pad_mdbas_edat-q loaded with 21976 rows
BarcelonaCiutat_SeccionsCensals loaded with 1068 rows
  Data_Referencia  Codi_Districte Nom_Districte  Codi_Barri Nom_Barri  AEB  \
0      2025-01-01               1  Ciutat Vella           1  el Raval    1   
1      2025-01-01               1  Ciutat Vella           1  el Raval    1   
2      2025-01-01               1  Ciutat Vella           1  el Raval    1   
3      2025-01-01               1  Ciutat Vella           1  el Raval    1   
4      2025-01-01               1  Ciutat Vella           1  el Raval    1   

   Seccio_Censal  Valor  NACIONALITAT_CONTINENT  SEXE  
0           1001   16.0                       1     1  
1           1001   41.0                       1     2  
2           1001   74.0                       2     1  
3           1001   68.0                       2     2  
4           1001  140.0                       3     1  
  Data_Referencia  Codi_Distric

## Read manually

In [25]:
# Rent
dfs["renda_bruta_llar"] = pd.read_csv(RAW_DATA_DIR/"2023_atles_renda_bruta_llar.csv")

# Data management (Code)

## Format census Tracks

In [ ]:
#Get data
census_tracks = dfs["BarcelonaCiutat_SeccionsCensals"].copy()

#Clean geometries
census_tracks["geometria_etrs89"] = census_tracks["geometria_etrs89"].apply(wkt.loads)
census_tracks = gpd.GeoDataFrame(census_tracks, geometry="geometria_etrs89", crs="EPSG:25831")

#Correct codi
census_tracks["Seccio_Censal"] = census_tracks["codi_districte"].astype(str).str.zfill(2) + census_tracks["codi_seccio_censal"].astype(str).str.zfill(3)
census_tracks["Seccio_Censal"]

#Keep relevant columns
census_tracks = census_tracks[['Seccio_Censal', 'geometria_etrs89']]
census_tracks.rename(columns={"geometria_etrs89": "geometry"}, inplace=True)
census_tracks.set_geometry("geometry", inplace=True)
census_tracks.columns



Index(['Seccio_Censal', 'geometry'], dtype='object')

In [57]:
census_tracks.head()

,Seccio_Censal,geometry
0,01001,"POLYGON ((431076.902 4581077.31, 431058.164 45..."
1,01002,"POLYGON ((431023.546 4581164.326, 430990.55 45..."
2,01003,"POLYGON ((430778.346 4580930.54, 430766.852 45..."
3,01004,"POLYGON ((430564.264 4581104.3, 430496.864 458..."
4,01005,"POLYGON ((430905.032 4581350.072, 430874.964 4..."


## Format data

In [ ]:
# Vehicles per type

#Get data
vehicles_type = dfs["Vehicles per type"].copy()

#Correct codi
vehicles_type["Seccio_Censal"] = vehicles_type["Codi_Districte"].astype(str).str.zfill(2) + vehicles_type["Seccio_censal"].astype(str).str.zfill(3)

#Keep relevant columns
vehicles_type = vehicles_type[['Seccio_Censal', 'Tipologia_parc', 'Nombre_vehicles']]

# Make wide format

vehicles_type = vehicles_type.pivot_table(index=['Seccio_Censal'], columns='Tipologia_parc', values='Nombre_vehicles', fill_value=0, ).reset_index()
vehicles_type.columns.name = None  # remove the pivoted column name
vehicles_type.head(2)

,Seccio_Censal,Altres vehicles,Camions,Ciclomotors,Furgonetes,Motos,Turismes
0,01001,111.0,47.0,245.0,37.0,146.0,349.0
1,01002,8.0,10.0,46.0,31.0,134.0,190.0


In [63]:
# Vehicles per type

#Get data
rent = dfs["renda_bruta_llar"].copy()

#Correct codi
rent["Seccio_Censal"] = rent["Codi_Districte"].astype(str).str.zfill(2) + rent["Seccio_Censal"].astype(str).str.zfill(3)

# #Keep relevant columns
rent = rent[['Seccio_Censal', 'Import_Renda_Bruta_€']]

rent.head(10)

,Seccio_Censal,Import_Renda_Bruta_€
0,01001,42658
1,01002,32672
2,01003,35938
3,01004,41877
4,01005,33004
5,01006,38981
6,01007,38052
7,01008,30205
8,01009,36196
9,01010,35587


In [ ]:
# Population by continent

#Get data
population = dfs["2025_pad_mdbas_nacionalitat-continent_sexe"].copy()

#Correct codi
population["Seccio_Censal"] = population["Seccio_Censal"].astype(str).str.zfill(5)

# #Keep relevant columns
population = population[['Seccio_Censal','Valor',	'NACIONALITAT_CONTINENT',	'SEXE']]

# # Make wide format
population_continent = population.pivot_table(index=['Seccio_Censal'], columns='NACIONALITAT_CONTINENT', values='Valor', fill_value=0, ).reset_index()
population_continent.columns.name = None  # remove the pivoted column name
population_continent.rename(columns={1:'Africa', 2:'America', 3:'Asia', 4:'Europe', 5:'Oceania'}, inplace=True)

population_gender = population.pivot_table(index=['Seccio_Censal'], columns='SEXE', values='Valor', fill_value=0, ).reset_index()
population_gender.columns.name = None  # remove the pivoted column name
population_gender.rename(columns={1:'Female', 2:'Male'}, inplace=True)

# population_continent.head(2)
population_gender.head()

,Seccio_Censal,Female,Male
0,01001,169.50,185.25
1,01002,158.75,177.00
2,01003,445.50,481.25
3,01004,358.75,412.50
4,01005,283.00,327.25


In [85]:
#Population

# Population by continent

#Get data
age = dfs["2025_pad_mdbas_edat-q"].copy()

#Correct codi
age["Seccio_Censal"] = age["Seccio_Censal"].astype(str).str.zfill(5)

# #Keep relevant columns
age = age[['Seccio_Censal','Valor','EDAT_Q']]

# # Make wide format
age = age.pivot_table(index=['Seccio_Censal'], columns='EDAT_Q', values='Valor', fill_value=0, ).reset_index()
age.columns.name = None  # remove the pivoted column name
age.rename(columns={
    0: '<5 years',
    1: '5-9 years',
    2: '10-14 years',
    3: '15-19 years',
    4: '20-24 years',
    5: '25-29 years',
    6: '30-34 years',
    7: '35-39 years',
    8: '40-44 years',
    9: '45-49 years',
    10: '50-54 years',
    11: '55-59 years',
    12: '60-64 years',
    13: '65-69 years',
    14: '70-74 years',
    15: '75-79 years',
    16: '80-84 years',
    17: '85-89 years',
    18: '90-94 years',
    19: '95-99 years',
    20: '100+'
}, inplace=True)
age.head()

,Seccio_Censal,<5 years,5-9 years,10-14 years,15-19 years,20-24 years,25-29 years,30-34 years,35-39 years,40-44 years,...,55-59 years,60-64 years,65-69 years,70-74 years,75-79 years,80-84 years,85-89 years,90-94 years,95-99 years,100+
0,01001,52.0,56.0,68.0,36.0,81.0,149.0,177.0,146.0,115.0,...,103.0,65.0,40.0,49.0,38.0,31.0,21.0,10.0,0.0,0.0
1,01002,51.0,38.0,36.0,35.0,103.0,196.0,182.0,143.0,120.0,...,62.0,45.0,59.0,49.0,27.0,20.0,20.0,14.0,0.0,0.0
2,01003,233.0,241.0,253.0,195.0,268.0,385.0,361.0,325.0,300.0,...,188.0,147.0,130.0,69.0,76.0,44.0,31.0,25.0,0.0,0.0
3,01004,133.0,100.0,112.0,138.0,240.0,323.0,395.0,337.0,275.0,...,183.0,127.0,89.0,71.0,54.0,38.0,21.0,18.0,0.0,0.0
4,01005,77.0,92.0,100.0,105.0,200.0,296.0,257.0,247.0,190.0,...,127.0,103.0,98.0,63.0,46.0,42.0,27.0,17.0,0.0,0.0


## Merge all data together

In [87]:
datasets = [census_tracks, vehicles_type, population_continent, population_gender, age, rent]
results = datasets[0]
for dataset in datasets[1:]:
    results = results.merge(dataset, on='Seccio_Censal', how='left')

# Plots

In [3]:
# Plots and visualizations

# Save results

In [90]:
#Save results and figures
results.to_parquet(INTERIM_DATA_DIR/"opendata.parquet")